# Description

This notebook reads all matrices from the MultiPLIER model (https://github.com/greenelab/multi-plier) trained in recount2, like gene loadings (Z) or the
latent space (B), and saves them into a Python friendly format (Pandas DataFrames in pickle format).

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
# pandas2ri.activate()

import settings as conf

In [3]:
readRDS = ro.r['readRDS']

# Read MultiPLIER model

In [4]:
conf.MULTIPLIER_SETTINGS['RECOUNT2_FULL_MODEL_FILE']

'/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data/recount_PLIER_model.RDS'

In [5]:
multiplier_full_model = readRDS(
    conf.MULTIPLIER_SETTINGS['RECOUNT2_FULL_MODEL_FILE']
)

# Matrix Z (loadings; genes x LVs)

In [6]:
multiplier_model_matrix = multiplier_full_model.rx2('Z')

In [7]:
multiplier_model_matrix

0.000000,0.000000,0.000000,...,0.003215,0.042435,0.287909


In [8]:
multiplier_model_matrix.rownames

'GAS6','MMP14','DSP',...,'PLEKHG6','GNGT2','SERPINH1'


In [9]:
multiplier_model_matrix.colnames

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f0ac23d6f40> [RTYPES.NILSXP]

In [10]:
with localconverter(ro.default_converter + pandas2ri.converter):
  multiplier_model_matrix_values = ro.conversion.rpy2py(multiplier_model_matrix)

In [11]:
multiplier_model_matrix_df = pd.DataFrame(
    data=multiplier_model_matrix_values,
    index=multiplier_model_matrix.rownames,
    columns=[f'LV{i}' for i in range(1, multiplier_model_matrix.ncol + 1)]
)

In [12]:
display(multiplier_model_matrix_df.shape)
assert multiplier_model_matrix_df.shape == (6750, 987)

(6750, 987)

In [13]:
multiplier_model_matrix_df.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


In [14]:
# make sure I'm seeing the same when loaded with R
assert multiplier_model_matrix_df.loc['GAS6', 'LV2'] == 0
assert multiplier_model_matrix_df.loc['GAS6', 'LV3'] == 0.039437739697954444
assert multiplier_model_matrix_df.loc['GAS6', 'LV984'] == 0.3473620915326928
assert multiplier_model_matrix_df.loc['GAS6', 'LV987'] == 0

assert multiplier_model_matrix_df.loc['SPARC', 'LV981'] == 0
assert multiplier_model_matrix_df.loc['SPARC', 'LV986'].round(8) == 0.12241734

## Save

In [15]:
output_dir = Path(
    conf.MULTIPLIER_SETTINGS['RECOUNT2_FULL_MODEL_FILE']
).resolve().parent
display(output_dir)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data')

In [16]:
output_file = Path(
    output_dir, 'multiplier_model_z.pkl'
).resolve()
display(output_file)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data/multiplier_model_z.pkl')

In [17]:
multiplier_model_matrix_df.to_pickle(output_file)

# Matrix B (latent space; LVs x samples)

In [18]:
multiplier_model_matrix = multiplier_full_model.rx2('B')

In [19]:
multiplier_model_matrix

-0.059297,0.006213,-0.026105,...,-0.104375,-0.015785,-0.024907


In [20]:
multiplier_model_matrix.rownames

"'1,REACTO...","'2,SVM Mo...","'3,REACTO...",...,"'985,IRIS...",'LV 986','LV 987'


In [21]:
multiplier_model_matrix.colnames

'SRP00059...,'SRP00059...,'SRP00059...,...,'SRP03559...,'SRP03559...,'SRP03559...


In [22]:
with localconverter(ro.default_converter + pandas2ri.converter):
  multiplier_model_matrix_values = ro.conversion.rpy2py(multiplier_model_matrix)

In [23]:
multiplier_model_matrix_df = pd.DataFrame(
    data=multiplier_model_matrix_values,
#    Look like the rows have a special meaning, so no overriding it.
#    index=[f'LV{i}' for i in range(1, multiplier_model_matrix.nrow + 1)],
    index=multiplier_model_matrix.rownames,
    columns=multiplier_model_matrix.colnames,
)

In [24]:
display(multiplier_model_matrix_df.shape)
assert multiplier_model_matrix_df.shape == (987, 37032)

(987, 37032)

In [25]:
multiplier_model_matrix_df.head()

,SRP000599.SRR013549,SRP000599.SRR013550,SRP000599.SRR013551,SRP000599.SRR013552,SRP000599.SRR013553,SRP000599.SRR013554,SRP000599.SRR013555,SRP000599.SRR013556,SRP000599.SRR013557,SRP000599.SRR013558,...,SRP035599.SRR1139372,SRP035599.SRR1139393,SRP035599.SRR1139388,SRP035599.SRR1139378,SRP035599.SRR1139399,SRP035599.SRR1139386,SRP035599.SRR1139375,SRP035599.SRR1139382,SRP035599.SRR1139356,SRP035599.SRR1139370
"1,REACTOME_MRNA_SPLICING",-0.059297,-0.047909,-0.049366,-0.065078,-0.036394,-0.046433,-0.040980,-0.040068,-0.046137,-0.048548,...,0.028220,0.035137,0.065077,0.078144,0.092362,0.069042,0.090914,0.096341,0.131115,0.171751
"2,SVM Monocytes",0.006213,0.003625,0.006605,0.009258,0.005061,0.004133,0.008950,0.007227,0.007241,0.005710,...,-0.050455,-0.034502,-0.033640,-0.049702,-0.037426,-0.050070,-0.022575,-0.055091,-0.056869,-0.018073
"3,REACTOME_TRANSMISSION_ACROSS_CHEMICAL_SYNAPSES",-0.026105,-0.032232,-0.020621,-0.027599,-0.035248,-0.038701,-0.032527,-0.030593,-0.028937,-0.027406,...,-0.028610,-0.033450,-0.030583,-0.032399,-0.029365,-0.025406,-0.033657,-0.031318,-0.030924,-0.027869
"4,REACTOME_NEURONAL_SYSTEM",-0.022080,-0.008971,-0.020342,-0.016260,-0.003023,0.002443,-0.020458,-0.023735,-0.021581,-0.022478,...,-0.037122,-0.029658,-0.036350,-0.039254,-0.035205,-0.036345,-0.034514,-0.035926,-0.040358,-0.031131
LV 5,0.007663,0.007036,0.006608,0.003446,0.006341,0.007106,0.007930,0.009164,0.008024,0.007938,...,-0.003056,-0.004784,-0.004352,-0.004160,-0.001085,-0.001884,-0.003561,-0.003546,-0.012107,-0.001193


In [26]:
# make sure I'm seeing the same when loaded with R
assert multiplier_model_matrix_df.loc['1,REACTOME_MRNA_SPLICING', 'SRP000599.SRR013549'].round(9) == -0.059296689
assert multiplier_model_matrix_df.loc['1,REACTOME_MRNA_SPLICING', 'SRP000599.SRR013553'].round(9) == -0.036394186

assert multiplier_model_matrix_df.loc['2,SVM Monocytes', 'SRP000599.SRR013549'].round(9) == 0.006212678
assert multiplier_model_matrix_df.loc['2,SVM Monocytes', 'SRP004637.SRR073776'].round(9) == -0.008800153

assert multiplier_model_matrix_df.loc['LV 9', 'SRP004637.SRR073774'].round(9) == 0.092318955
assert multiplier_model_matrix_df.loc['LV 9', 'SRP004637.SRR073776'].round(9) == 0.100114294

## Make sure no GTEx samples are included

In [27]:
# Test search string first
_tmp = multiplier_model_matrix_df.columns.str.contains('SRP000599.', regex=False)
assert _tmp[0]
assert _tmp[1]
assert not _tmp[-1]

In [28]:
GTEX_ACCESSION_CODE = 'SRP012682'

In [29]:
_tmp = multiplier_model_matrix_df.columns.str.contains(GTEX_ACCESSION_CODE, regex=False)
assert not _tmp.any()

## Save

In [30]:
output_dir = Path(
    conf.MULTIPLIER_SETTINGS['RECOUNT2_FULL_MODEL_FILE']
).resolve().parent
display(output_dir)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data')

In [31]:
output_file = Path(output_dir, 'multiplier_model_b.pkl').resolve()
display(output_file)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data/multiplier_model_b.pkl')

In [32]:
multiplier_model_matrix_df.to_pickle(output_file)

# Matrix U (gene sets x LVs)

In [33]:
multiplier_model_matrix = multiplier_full_model.rx2('U')

In [34]:
multiplier_model_matrix

0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000


In [35]:
multiplier_model_matrix.rownames

'IRIS_Bce...,'IRIS_Bce...,'IRIS_Bce...,...,'PID_BCR_...,'PID_TELO...,'PID_PI3K...


In [36]:
multiplier_model_matrix.colnames

'LV1','LV2','LV3',...,'LV985','LV986','LV987'


In [37]:
with localconverter(ro.default_converter + pandas2ri.converter):
  multiplier_model_matrix_values = ro.conversion.rpy2py(multiplier_model_matrix)

In [38]:
multiplier_model_matrix_df = pd.DataFrame(
    data=multiplier_model_matrix_values,
    index=multiplier_model_matrix.rownames,
    columns=multiplier_model_matrix.colnames,
)

In [39]:
display(multiplier_model_matrix_df.shape)
assert multiplier_model_matrix_df.shape == (628, 987)

(628, 987)

In [40]:
multiplier_model_matrix_df.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-Memory_IgM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-naive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-N0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-Th1-restimulated12hour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
# make sure I'm seeing the same when loaded with R
assert multiplier_model_matrix_df.loc['IRIS_Bcell-Memory_IgG_IgA', 'LV1'] == 0
assert multiplier_model_matrix_df.loc['IRIS_Bcell-Memory_IgG_IgA', 'LV898'].round(7) == 0.5327689
assert multiplier_model_matrix_df.loc['IRIS_Bcell-Memory_IgG_IgA', 'LV977'].round(7) == 0.1000158
assert multiplier_model_matrix_df.loc['IRIS_Bcell-Memory_IgG_IgA', 'LV986'] == 0
assert multiplier_model_matrix_df.loc['IRIS_Bcell-Memory_IgG_IgA', 'LV987'] == 0

assert multiplier_model_matrix_df.loc['IRIS_Bcell-naive', 'LV851'].round(8) == 0.01330388
assert multiplier_model_matrix_df.loc['IRIS_Bcell-naive', 'LV977'].round(7) == 0.3966446

## Save

In [42]:
output_dir = Path(
    conf.MULTIPLIER_SETTINGS['RECOUNT2_FULL_MODEL_FILE']
).resolve().parent
display(output_dir)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data')

In [43]:
output_file = Path(
    output_dir, 'multiplier_model_u.pkl'
).resolve()
display(output_file)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data/multiplier_model_u.pkl')

In [44]:
multiplier_model_matrix_df.to_pickle(output_file)

# Matrix U - AUC

In [45]:
multiplier_model_matrix = multiplier_full_model.rx2('Uauc')

In [46]:
multiplier_model_matrix

0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000


In [47]:
multiplier_model_matrix.rownames

'IRIS_Bce...,'IRIS_Bce...,'IRIS_Bce...,...,'PID_BCR_...,'PID_TELO...,'PID_PI3K...


In [48]:
multiplier_model_matrix.colnames

'LV1','LV2','LV3',...,'LV985','LV986','LV987'


In [49]:
with localconverter(ro.default_converter + pandas2ri.converter):
  multiplier_model_matrix_values = ro.conversion.rpy2py(multiplier_model_matrix)

In [50]:
multiplier_model_matrix_df = pd.DataFrame(
    data=multiplier_model_matrix_values,
    index=multiplier_model_matrix.rownames,
    columns=multiplier_model_matrix.colnames,
)

In [51]:
display(multiplier_model_matrix_df.shape)
assert multiplier_model_matrix_df.shape == (628, 987)

(628, 987)

In [52]:
multiplier_model_matrix_df.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
IRIS_Bcell-Memory_IgG_IgA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-Memory_IgM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_Bcell-naive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-N0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IRIS_CD4Tcell-Th1-restimulated12hour,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# make sure I'm seeing the same when loaded with R
assert multiplier_model_matrix_df.loc['PID_FASPATHWAY', 'LV136'] == 0
assert multiplier_model_matrix_df.loc['PID_INTEGRIN1_PATHWAY', 'LV136'].round(7) == 0.8832853
assert multiplier_model_matrix_df.loc['REACTOME_COLLAGEN_FORMATION', 'LV136'].round(7) == 0.8707412

assert multiplier_model_matrix_df.loc['PID_FASPATHWAY', 'LV603'] == 0
assert multiplier_model_matrix_df.loc['IRIS_Neutrophil-Resting', 'LV603'].round(7) == 0.9057506
assert multiplier_model_matrix_df.loc['SVM Neutrophils', 'LV603'].round(7) == 0.9797889

### Save

In [54]:
output_dir = Path(
    conf.MULTIPLIER_SETTINGS['RECOUNT2_FULL_MODEL_FILE']
).resolve().parent
display(output_dir)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data')

In [55]:
output_file = Path(
    output_dir, 'multiplier_model_u_auc.pkl'
).resolve()
display(output_file)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data/multiplier_model_u_auc.pkl')

In [56]:
multiplier_model_matrix_df.to_pickle(output_file)

# Model metadata

In [57]:
model_names = list(multiplier_full_model.names)
display(model_names)

['residual',
 'B',
 'Z',
 'U',
 'C',
 'numActPath',
 'L1',
 'L2',
 'L3',
 'heldOutGenes',
 'withPrior',
 'Uauc',
 'Up',
 'summary']

In [58]:
with localconverter(ro.default_converter + pandas2ri.converter):
    model_metadata = {k: ro.conversion.rpy2py(multiplier_full_model.rx2(k))[0] for k in ('L1', 'L2', 'L3')}

In [59]:
model_metadata

{'L1': 120.5660870071812, 'L2': 241.1321740143624, 'L3': 0.012696760842460974}

In [60]:
assert len(model_metadata) == 3

In [61]:
assert model_metadata['L2'] == 241.1321740143624

## Save

In [62]:
output_dir = Path(
    conf.MULTIPLIER_SETTINGS['RECOUNT2_FULL_MODEL_FILE']
).resolve().parent
display(output_dir)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data')

In [63]:
output_file = Path(
    output_dir, 'multiplier_model_metadata.pkl'
).resolve()
display(output_file)

PosixPath('/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data/multiplier_model_metadata.pkl')

In [64]:
with open(output_file, 'wb') as handle:
    pickle.dump(model_metadata, handle, protocol=pickle.HIGHEST_PROTOCOL)